In [47]:
import pandas as pd

from backtesting import Strategy, Backtest

from backtesting.lib import crossover

import talib
import pandas_ta

import warnings
import strategies

warnings.filterwarnings("ignore")

In [48]:
ticker = "SBIN"
df = pd.read_parquet(f"../res/data/{ticker}.parquet")
df = df.loc['2024':]

In [49]:
def crossup(open, close, value):
    return open <= value and value <= close


def crossdown(open, close, value):
    return open >= value and value >= close

In [50]:
class SupertrendStrategy(Strategy):
    length = 10

    def supertrend(self, df, length):
        supertrend = pandas_ta.supertrend(df.High, df.Low, df.Close, length=length)
        return supertrend.to_numpy().T[1]

    def init(self):
        df = self.data.df
        self.dir = self.I(self.supertrend, df, self.length)

    def next(self):
        if self.dir[-1] * self.dir[-2] > 0:
            return

        if self.dir[-1] > 0:
            self.buy()
        elif self.dir[-1] < 0:
            self.sell()

## Optimization


### Training


In [51]:
bt = Backtest(
    df.iloc[: len(df)//2],
    SupertrendStrategy,
    cash=50_000,
    commission=20 / 50_000,
    exclusive_orders=True,
)
bt.run()
#stats = bt.optimize(length=[x for x in range(5, 32, 2)], maximize="Sharpe Ratio")
#print("Strategy:", stats._strategy)
#print("Sharpe ratio:", stats["Sharpe Ratio"])

bt.plot()

GridPlot(id='p7600', ...)

### Testing


In [52]:
bt = Backtest(
    df.iloc[len(df)//2:],
    SupertrendStrategy,
    cash=50_000,
    commission=20 / 50_000,
    exclusive_orders=True,
)
print(bt.run(length=stats._strategy.length))
bt.plot()

Start                     2024-01-23 05:20:00
End                       2024-02-15 06:50:00
Duration                     23 days 01:30:00
Exposure Time [%]                   98.934426
Equity Final [$]                   58755.9587
Equity Peak [$]                    58906.1087
Return [%]                          17.511917
Buy & Hold Return [%]               21.398781
Return (Ann.) [%]                  711.575164
Volatility (Ann.) [%]              221.351881
Sharpe Ratio                         3.214679
Sortino Ratio                       58.179934
Calmar Ratio                       154.097264
Max. Drawdown [%]                   -4.617701
Avg. Drawdown [%]                   -0.763129
Max. Drawdown Duration        7 days 20:25:00
Avg. Drawdown Duration        0 days 14:40:00
# Trades                                   24
Win Rate [%]                             50.0
Best Trade [%]                       9.211692
Worst Trade [%]                       -1.4416
Avg. Trade [%]                    

GridPlot(id='p7980', ...)

## Comparison of Strategies


In [53]:
strats = [
    strategies.SupertrendStrategy,
    strategies.BBandReversion,
    strategies.MACDCross,
    strategies.EMACross,
    strategies.VWAPCross,
]


def backtest_strategies(df, strategies):
    res = []
    colname = []
    for strat in strategies:
        bt = Backtest(df, strat, cash=10_000, exclusive_orders=True)
        strat_name = bt._strategy.__name__
        res.append(bt.run().to_frame())
        colname.append(strat_name)
        bt.plot(
            filename="../res/backtests/" + strat_name + ".html",
            # resample="5D",
            smooth_equity=True,
            open_browser=False,
        )
    res = pd.concat(res, axis=1)
    res.columns = colname
    return res


backtest_strategies(df, strats)

,SupertrendStrategy,BBandReversion,MACDCross,EMACross,VWAPCross
Start,2024-01-01 03:45:00,2024-01-01 03:45:00,2024-01-01 03:45:00,2024-01-01 03:45:00,2024-01-01 03:45:00
End,2024-02-15 06:50:00,2024-02-15 06:50:00,2024-02-15 06:50:00,2024-02-15 06:50:00,2024-02-15 06:50:00
Duration,45 days 03:05:00,45 days 03:05:00,45 days 03:05:00,45 days 03:05:00,45 days 03:05:00
Exposure Time [%],97.088971,97.088971,98.564986,95.407954,91.676917
Equity Final [$],10815.35,11365.65,10289.35,10941.85,10565.4295
Equity Peak [$],10842.65,11394.9,10314.7,10969.15,10791.667
Return [%],8.1535,13.6565,2.8935,9.4185,5.654295
Buy & Hold Return [%],17.561165,17.561165,17.561165,17.561165,17.561165
Return (Ann.) [%],75.11482,155.808394,32.53289,98.843424,44.57737
Volatility (Ann.) [%],45.07816,77.31205,34.052242,58.601664,36.960696
